In [12]:
import os

In [13]:
%cd "C:\Users\shrey\OneDrive\Desktop\Py-Projects\Text-Summerization-Tool\research"

C:\Users\shrey\OneDrive\Desktop\Py-Projects\Text-Summerization-Tool\research


In [14]:
%pwd

'C:\\Users\\shrey\\OneDrive\\Desktop\\Py-Projects\\Text-Summerization-Tool\\research'

In [15]:
os.chdir("../")

In [16]:
%pwd

'C:\\Users\\shrey\\OneDrive\\Desktop\\Py-Projects\\Text-Summerization-Tool'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [18]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
import zipfile
from textSummarization.logging import logger
from textSummarization.utils.common import get_size

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File  of size: {get_size(Path(self.config.local_data_file))} already exists!")  

        
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-30 12:06:38,210: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-30 12:06:38,214: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-30 12:06:38,217: INFO: common: directory: artifacts created successfully]
[2024-01-30 12:06:38,221: INFO: common: directory: artifacts/data_ingestion created successfully]
[2024-01-30 12:07:18,338: INFO: 739868373: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BDCC:11AF:37DD48:441B3B:65B8981E
Accept-Ranges: bytes
Date: Tue, 30 Jan 2024 06:36:39 GMT
Via: 1.1 varnish
X-Served-By: cache-maa

EOFError: 